# Data Cleaning & Feature Engineering (Day 2)

This notebook prepares a clean and analysis-ready transaction dataset
by performing minimal data cleaning and constructing features that
will be used for both SQL analytics and machine learning models.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/raw/PS_20174392719_1491204439457_log.csv')

No missing values were found in the dataset. 
Therefore, no imputation was required.

In [ ]:
#handle outlier

In [3]:
df['amount_log'] = np.log1p(df['amount'])

In [4]:
df['amount_bucket'] = pd.cut(
    df['amount'],
    bins=[0,100,1000,10000,100000,np.inf],
    labels=['very_small','small','medium','large','very_large']
)

In [5]:
df['balance_mismatch_orig'] = (
    df['oldbalanceOrg'] - df['amount'] != df['newbalanceOrig']
)

In [6]:
df['balance_mismatch_dest'] = (
    df['oldbalanceDest'] + df['amount'] != df['newbalanceDest']
)

In [ ]:
#step -> hour,day

In [7]:
df['hour'] = (df['step'] - 1) % 24
df['day'] = (df['step'] - 1) // 24 + 1

In [ ]:
# handle type

In [8]:
df['type']

0           PAYMENT
1           PAYMENT
2          TRANSFER
3          CASH_OUT
4           PAYMENT
             ...   
6362615    CASH_OUT
6362616    TRANSFER
6362617    CASH_OUT
6362618    TRANSFER
6362619    CASH_OUT
Name: type, Length: 6362620, dtype: object

In [ ]:
#creat new dataset

In [9]:
final_cols = [
    # identifiers
    'step', 'day', 'hour', 'type',
    
    # transaction
    'amount', 'amount_log', 'amount_bucket',
    
    # balances
    'oldbalanceOrg', 'newbalanceOrig',
    'oldbalanceDest', 'newbalanceDest',
    'balance_mismatch_orig', 'balance_mismatch_dest',
    
    # labels
    'isFraud', 'isFlaggedFraud'
]


In [10]:
transactions_clean = df[final_cols].copy()

In [12]:
if 'amount_bucket' in transactions_clean.columns:
    transactions_clean['amount_bucket'] = transactions_clean['amount_bucket'].astype('string')

for c in ['balance_mismatch_orig', 'balance_mismatch_dest']:
    if c in transactions_clean.columns:
        transactions_clean[c] = transactions_clean[c].astype('int8')

transactions_clean.to_parquet(
    "../data/processed/transactions_clean.parquet",
    index=False,
    engine="pyarrow"
)

## Day 2 Summary

- No missing values required imputation.
- Transaction amount features were transformed using log scaling and bucketing.
- Balance mismatch indicators were created as potential fraud signals.
- Time-based features (hour, day) were derived from the global step variable.
- A clean and analysis-ready dataset (`transactions_clean`) was exported for downstream SQL analytics and modeling.